<a href="https://colab.research.google.com/github/satz2901/congenial-barnacle/blob/main/Dogs_vs_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras=tf.keras

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
(raw_train,raw_validation,raw_test),metadata=tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]','train[80%:90%]','train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [24]:
IMG_SIZE=160
def format_example(image,label):
   image=tf.cast(image,tf.float32)
   image=(image/127.5)-1
   image=tf.image.resize(image,(IMG_SIZE,IMG_SIZE))
   return image,label


In [25]:
train=raw_train.map(format_example)
validation=raw_validation.map(format_example)
test=raw_test.map(format_example)

In [28]:
BATCH_SIZE=32
SHUFFLE_BUFFER_SIZE=1000
train_batches=train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches=validation.batch(BATCH_SIZE)
test_batches=test.batch(BATCH_SIZE)

In [ ]:
for img,label in raw_train.take(2):
  print("original shape:",img.shape)

for img,label in train.take(2):
  print("new shape:",img.shape)

In [ ]:
IMG_SHAPE=(IMG_SIZE,IMG_SIZE,3)

base_model=tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [ ]:
for image, _ in train_batches.take(1):
   pass

feature_batch = base_model(image)
print(feature_batch.shape)

In [34]:
base_model.trainable=False

In [36]:
global_average_layer=tf.keras.layers.GlobalAveragePooling2D()

In [37]:
prediction_layer=tf.keras.layers.Dense(1)

In [38]:
model=tf.keras.Sequential([base_model,
                           global_average_layer,
                           prediction_layer])

In [40]:
base_learning_rate=0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
initial_epoch=3
validation_steps=20
loss0,accuracy0=model.evaluate(validation_batches,steps=validation_steps)

In [ ]:
history= model.fit(train_batches,
                   epochs=initial_epoch,
                   validation_data=validation_batches)
acc=history.history['accuracy']
print(acc)

In [44]:
model.save("dogs_vs_cats.h5")

new_model= tf.keras.models.load_model('dogs_vs_cats.h5')